In [132]:
from torchvision.datasets import ImageFolder
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import cv2
import numpy as np
import pandas as pd

transform = transforms.Compose([
    transforms.Resize((50,50)), 
    transforms.ToTensor() 
])

data = ImageFolder(root="./data", transform=transform)

In [133]:
data_loader = DataLoader(data, batch_size=10, shuffle=True)

In [134]:
images, labels = next(iter(data_loader))
print(images.shape, labels.shape) # shape : [개수, 채널 수, 높이, 너비]

torch.Size([10, 3, 50, 50]) torch.Size([10])


In [135]:
class Mob_Dog(nn.Module): # 클래스 생성
    def __init__(self):
        super(Mob_Dog, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 9 * 9, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x))) # 두번 컨볼루션 연산 해주기
        
        
        
        x = x.view(-1, 16 * 9 * 9)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [136]:
# 모델 객체 생성, 옵티마이저 생성
model = Mob_Dog()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [137]:
from tqdm import tqdm # 프로그레스 바 생성
from torch.optim.lr_scheduler import ReduceLROnPlateau # 스케줄러 생성 
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3, verbose=True)


for epoch in range(10): # 10회 반복 
    running_loss = 0.0 # 1 epoch당 누적 로스값 
    #==================================================================================
    pbar = tqdm(enumerate(data_loader), total=len(data_loader)) 
    for i, data in pbar: # pbar가 데이터 로더 역할 수행
        inputs, labels = data
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() # 로스 누적 
        if i % 10 == 9:
            pbar.set_description(f'Epoch [{epoch + 1}/{10}], 횟수 [{i + 1}/{len(data_loader)}], Loss: {running_loss / 10:.4f}')
            running_loss = 0.0
    #==================================================================================   
    # 모델 평가
    correct = 0
    total = 0
    with torch.no_grad():
        for data in data_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1) # dim=1을 주어 최종 값 계산
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    epoch_accuracy = correct / total
    print(f'{epoch} Epoch 종료 후 train_score : {(100 * epoch_accuracy)}')
    
    
    scheduler.step(loss)
    if scheduler.num_bad_epochs >= scheduler.patience:
        print(f"Early Stopping at : {epoch} Epoch")
        break

Epoch [1/10], 횟수 [10/18], Loss: 0.6937: 100%|██████████| 18/18 [00:00<00:00, 47.40it/s]


0 Epoch 종료 후 train_score : 51.13636363636363


Epoch [2/10], 횟수 [10/18], Loss: 0.6928: 100%|██████████| 18/18 [00:00<00:00, 49.38it/s]


1 Epoch 종료 후 train_score : 51.13636363636363


Epoch [3/10], 횟수 [10/18], Loss: 0.6916: 100%|██████████| 18/18 [00:00<00:00, 51.71it/s]


2 Epoch 종료 후 train_score : 51.13636363636363


Epoch [4/10], 횟수 [10/18], Loss: 0.6920: 100%|██████████| 18/18 [00:00<00:00, 52.14it/s]


3 Epoch 종료 후 train_score : 51.13636363636363


Epoch [5/10], 횟수 [10/18], Loss: 0.6942: 100%|██████████| 18/18 [00:00<00:00, 52.33it/s]


4 Epoch 종료 후 train_score : 51.13636363636363
Early Stopping at : 4 Epoch


In [138]:
# 모델 저장
torch.save(model.state_dict(), 'mob_dog.pth')

# 모델 로드
model = Mob_Dog()
model.load_state_dict(torch.load('mob_dog.pth'))
model.eval()

# 이미지 예측
img = cv2.imread('./data/mop_dog/0.jpg')
img = cv2.resize(img, (50, 50))
#img = img / 255.0
img = np.transpose(img, (2, 0, 1))
img = torch.Tensor(img)
img = img.unsqueeze(0)

with torch.no_grad():
    output = model(img)
    _, predicted = torch.max(output.data, 1)
predicted # 이런 젠장, 걸레라고 나온다 

tensor([1])